In [13]:
import json
import folium
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.colors import LogNorm  # 작은 차이도 잘 보이게 로그 스케일 적용

In [14]:
# -----------------------------
# 1. CSV 불러오기
# -----------------------------
hangang_df = pd.read_csv("../../../measure/ACC/results/hangang_population_weighted.csv")
nonhangang_df = pd.read_csv("../../../measure//ACC/results/nonhangang_population_weighted_district.csv")

In [15]:
# -----------------------------
# 2. GeoDataFrame 불러오기
# -----------------------------
gu = gpd.read_file("../../../map/result/gu.gpkg", layer="gu").to_crs(4326)
parks = gpd.read_file("../../../map/result/parks.gpkg", layer="hangang_parks").to_crs(4326)

In [16]:
# -----------------------------
# 3. merge
# -----------------------------
# 자치구별 종합점수
gu = gu.merge(
    nonhangang_df[["District", "인구가중점수_std_mean"]],
    left_on="SIGUNGU_NM",
    right_on="District",
    how="left"
).drop(columns=["District"])

# 한강공원: park_name 영문 → 한글 변환
park_name_map = {
    "Banpo": "반포한강공원", "Gangseo": "강서한강공원", "Gwangnaru": "광나루한강공원",
    "Ichon": "이촌한강공원", "Jamsil": "잠실한강공원", "Jamwon": "잠원한강공원",
    "Mangwon": "망원한강공원", "Nanji": "난지한강공원", "Ttukseom": "뚝섬한강공원",
    "Yanghwa": "양화한강공원", "Yeouido": "여의도한강공원"
}
parks["park_name"] = parks["park_name"].map(park_name_map)

# 한강공원 종합점수 merge
parks = parks.merge(
    hangang_df[["공원명", "인구가중점수_std"]],
    left_on="park_name",
    right_on="공원명",
    how="left"
).drop(columns=["공원명"])

In [17]:
# -----------------------------
# 4. colormap (구 + 공원 전체 기준)
# -----------------------------
all_min = min(gu["인구가중점수_std_mean"].min(), parks["인구가중점수_std"].min())
all_max = max(gu["인구가중점수_std_mean"].max(), parks["인구가중점수_std"].max())

cmap = plt.cm.Purples
norm = colors.Normalize(vmin=all_min, vmax=all_max)

In [18]:
# -----------------------------
# 5. folium 지도 생성
# -----------------------------
m = folium.Map(location=[37.5665,126.9780], zoom_start=11, tiles="CartoDB positron")

# 자치구 레이어
folium.GeoJson(
    gu,
    style_function=lambda f: {
        "fillColor": "#eeeeee" if pd.isna(f["properties"]["인구가중점수_std_mean"]) else 
                     colors.rgb2hex(cmap(norm(f["properties"]["인구가중점수_std_mean"]))),
        "color": "#666",
        "weight": 1,
        "fillOpacity": 0.6
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["SIGUNGU_NM", "인구가중점수_std_mean"],
        aliases=["자치구", "종합점수"]
    )
).add_to(m)

# 한강공원 레이어
folium.GeoJson(
    parks,
    name="한강공원",
    style_function=lambda f: {
        "fillColor": "#eeeeee" if pd.isna(f["properties"]["인구가중점수_std"]) else 
                     colors.rgb2hex(cmap(norm(f["properties"]["인구가중점수_std"]))),
        "color": "#666",
        "weight": 1.2,
        "fillOpacity": 0.8
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["park_name", "인구가중점수_std"],
        aliases=["공원", "종합점수"]
    )
).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

In [19]:
# 저장
m.save("../../result/ACC/population_weighted_visualization.html")
print("population_weighted_visualization.html 저장 완료")

population_weighted_visualization.html 저장 완료
